# Quick example of WL Classification
## using Valen's warm up and our virtual label model
This example tries to show the usage of the classes and methods
proposed here to show classification with weakly labeled datasets.

We will use the model proposed by Valen and their warm up prior to 20 epochs
with our method (We will use the

In [1]:
# Importing general libraries
import torch
import torch.nn as nn
import torchvision
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Importing specific classes and functions
from utils.weakener import Weakener

#Importing drawing tools
import matplotlib.pyplot as plt

#importing dataloaders
from datasets.openml_datasets import OpenML_Dataset
from datasets.torch_datasets import Torch_Dataset
from models.model import MLP,mlp_valen
from utils.trainig_testing import train_model,evaluate_model,train_and_evaluate
from utils.losses import EMLoss, CELoss, PartialLoss,OSLCELoss


In [2]:
# DS stores the dataset and its related attributes
DS = Torch_Dataset('mnist')
train_x, train_y, test_x, test_y = DS.get_data()
# WL stores pocesses relative to the Weakening process
WL = Weakener(DS.num_classes)


In [3]:
# Generation of the mixing matrix according to the model
#  we've chosen.
WL.generate_M(model_class='pll')

In [4]:
# We generate the weak labels (this step is optional as WL.virtual_labels(train_y) can do it)
z, w = WL.generate_weak(train_y)

z is the numerical encodig of the weak label while w is a one-hot
representation of that variable. z also encodes the row of the M matrix
for that given weak label.

Let's consider the example of partial label learning for the iris dataset.
labels are encoded as ```{0: '011', 1: '101', 2: '110', 3: '111'}```
so having an isntance with `z=3` means `w=[1,1,1]`, i.e., the weak label contains every label.

In [5]:
# We generate the weak labels (this step is optional as WL.virtual_labels(train_y) can do it)
WL.virtual_matrix()
#And we generate the virtual labels
WL.virtual_labels(train_y)


#we make a new dataloader because valen takes the weak label instead of the
# virtual label.
indices = torch.arange(0,len(train_x))
valen_train_data = TensorDataset(train_x, w ,train_y, indices)
valen_train_loader = DataLoader(valen_train_data, batch_size=64, shuffle=True)


WL.virtual_labels(train_y)
DS.include_weak(WL.v)
# Once we have our dataset with all the labels we need we can create the dataloaders.
trainloader, testloader = DS.get_dataloader()

In [6]:
mlp = MLP(input_size=DS.num_features, hidden_sizes=[DS.num_features,DS.num_features,DS.num_features],
          output_size=DS.num_classes, dropout_p=0)

In [7]:
#optimizer = torch.optim.SGD(mlp.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)
loss_fn_bk = OSLCELoss()

_,_,mlp =  train_model(mlp,valen_train_loader, optimizer, loss_fn_bk,num_epochs=10, return_model=True)

Epoch 1/10 - Loss: 1.2057 - Accuracy: 0.7010
Epoch 2/10 - Loss: 0.1800 - Accuracy: 0.9153
Epoch 3/10 - Loss: 0.1334 - Accuracy: 0.9358
Epoch 4/10 - Loss: 0.1153 - Accuracy: 0.9429
Epoch 5/10 - Loss: 0.1015 - Accuracy: 0.9497
Epoch 6/10 - Loss: 0.0928 - Accuracy: 0.9519
Epoch 7/10 - Loss: 0.0952 - Accuracy: 0.9523
Epoch 8/10 - Loss: 0.0778 - Accuracy: 0.9592
Epoch 9/10 - Loss: 0.0904 - Accuracy: 0.9562
Epoch 10/10 - Loss: 0.0771 - Accuracy: 0.9604


In [8]:
#Lets try with the values predicted from that model
_,train_y_new = torch.max(nn.functional.softmax(mlp(train_x),dim=1),dim=1)
print(train_y_new)
train_y_new = torch.eye(DS.num_classes)[train_y_new]
print(train_y_new.shape)
WL2 = Weakener(DS.num_classes)
WL2.generate_M(model_class='pll',pll_p=0.5)

z2, w2 = WL2.generate_weak(train_y_new)
WL2.virtual_matrix()
#And we generate the virtual labels
WL2.virtual_labels(train_y_new)
indices = torch.arange(0,len(train_x))
print(train_x.shape)
print(torch.tensor(WL2.v).shape)
print(train_y_new.shape)
print(indices.shape)

new_train_data = TensorDataset(train_x, torch.tensor(WL2.v) ,train_y_new, indices)
new_train_loader = DataLoader(new_train_data, batch_size=64, shuffle=True)

new_test_data = TensorDataset(test_x, test_y)
new_test_loader = DataLoader(new_test_data, batch_size=64, shuffle=True)

mlp2 = MLP(input_size=DS.num_features, hidden_sizes=[DS.num_features,DS.num_features,DS.num_features],
          output_size=DS.num_classes, dropout_p=0.5, bn=True)

tensor([5, 0, 4,  ..., 5, 6, 8])
torch.Size([60000, 10])
torch.Size([60000, 784])
torch.Size([60000, 10])
torch.Size([60000, 10])
torch.Size([60000])


In [11]:
# Define the optimizer and loss function
mlp2 = MLP(input_size=DS.num_features, hidden_sizes=[DS.num_features,DS.num_features,DS.num_features],
          output_size=DS.num_classes, dropout_p=0.5, bn=True)
loss_fn = CELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)
mlp2, results = train_and_evaluate(mlp2, new_train_loader, new_test_loader,
                                  optimizer, loss_fn, num_epochs=30)

# Print the results
print('Train Loss:', results['train_loss'][-1])
print('Train Accuracy:', results['train_acc'][-1])
print('Test Accuracy:', results['test_acc'][-1])

Epoch 1/30: Train Loss: 3.1698, Train Acc: 0.1047, Test Acc: 0.1031
Epoch 2/30: Train Loss: 3.1705, Train Acc: 0.1035, Test Acc: 0.1018
Epoch 3/30: Train Loss: 3.1536, Train Acc: 0.1037, Test Acc: 0.1013
Epoch 4/30: Train Loss: 3.1658, Train Acc: 0.1042, Test Acc: 0.1029
Epoch 5/30: Train Loss: 3.1712, Train Acc: 0.1017, Test Acc: 0.1016
Epoch 6/30: Train Loss: 3.1646, Train Acc: 0.1037, Test Acc: 0.1016
Epoch 7/30: Train Loss: 3.1763, Train Acc: 0.1034, Test Acc: 0.1021
Epoch 8/30: Train Loss: 3.1468, Train Acc: 0.1040, Test Acc: 0.0980
Epoch 9/30: Train Loss: 3.1529, Train Acc: 0.1039, Test Acc: 0.1003
Epoch 10/30: Train Loss: 3.1626, Train Acc: 0.1026, Test Acc: 0.1007
Epoch 11/30: Train Loss: 3.1652, Train Acc: 0.1043, Test Acc: 0.0989
Epoch 12/30: Train Loss: 3.1867, Train Acc: 0.1016, Test Acc: 0.1024
Epoch 13/30: Train Loss: 3.1801, Train Acc: 0.1032, Test Acc: 0.0994
Epoch 14/30: Train Loss: 3.1527, Train Acc: 0.1022, Test Acc: 0.1037
Epoch 15/30: Train Loss: 3.1664, Train Acc:

KeyboardInterrupt: 

In [ ]:
figure, axis = plt.subplots(1, 2)

# For Sine Function
axis[0].plot(results['train_loss'])
axis[0].set_title("Train_loss")

# For Cosine Function
axis[1].plot(results['train_acc'])
axis[1].plot(results['test_acc'])
axis[1].set_title("Accuracy")
plt.show()

In [ ]:
#We need to include this Weak Labels into the dataset,
# we need to include w to mantain the coherence. z will only be used
# in the loss for the EM
DS.include_weak(WL.v)
# Once we have our dataset with all the labels we need we can create the dataloaders.
trainloader, testloader = DS.get_dataloader()

print(len(testloader.dataset))
print(len(trainloader.dataset))

Now we have a trainloader containing (X,v,y) and a testloader containig (X,y)
so we can just establish our model and train it.

In [ ]:
# Create an instance of the MLP class
mlp = MLP(input_size=DS.num_features, hidden_sizes=[20],
          output_size=DS.num_classes, dropout_p=0.5)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)
loss_fn = CELoss()

train_losses,train_accs=train_model(mlp,trainloader,optimizer,
                                    loss_fn,num_epochs=10)

# Print the final training loss and accuracy
print('Final Training Loss: {:.4f}'.format(train_losses[-1]))
print('Final Training Accuracy: {:.4f}'.format(train_accs[-1]))

In [ ]:
figure, axis = plt.subplots(1, 2)

# For Sine Function
axis[0].plot(train_losses)
axis[0].set_title("Train_loss")

# For Cosine Function
axis[1].plot(train_accs)
axis[1].set_title("Train_accuracy")
plt.show()

In [ ]:
# Create an instance of the MLP class
mlp = MLP(input_size=DS.num_features, hidden_sizes=[20],
          output_size=DS.num_classes, dropout_p=0.5)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)
loss_fn = CELoss()

# Train and evaluate the MLP on the training data, test data, and get the results
mlp, results = train_and_evaluate(mlp, trainloader, testloader,
                                  optimizer, loss_fn, num_epochs=10)

# Print the results
print('Train Loss:', results['train_loss'][-1])
print('Train Accuracy:', results['train_acc'][-1])
print('Test Accuracy:', results['test_acc'][-1])

In [ ]:
figure, axis = plt.subplots(1, 2)

# For Sine Function
axis[0].plot(results['train_loss'])
axis[0].set_title("Train_loss")

# For Cosine Function
axis[1].plot(results['train_acc'])
axis[1].plot(results['test_acc'])
axis[1].set_title("Accuracy")
plt.show()